In [114]:
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [115]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [116]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]


In [117]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M5
start_datetime = datetime(2024,12,15)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

,time,open,high,low,close
0,2024-12-16 00:00:00,1.04850,1.04863,1.04797,1.04856
1,2024-12-16 00:05:00,1.04856,1.04865,1.04854,1.04856
2,2024-12-16 00:10:00,1.04856,1.04871,1.04853,1.04862
3,2024-12-16 00:15:00,1.04862,1.04880,1.04857,1.04867
4,2024-12-16 00:20:00,1.04871,1.04876,1.04857,1.04864
...,...,...,...,...,...
41036,2025-07-04 11:40:00,1.17735,1.17764,1.17714,1.17756
41037,2025-07-04 11:45:00,1.17756,1.17760,1.17720,1.17740
41038,2025-07-04 11:50:00,1.17740,1.17760,1.17726,1.17754
41039,2025-07-04 11:55:00,1.17754,1.17764,1.17736,1.17746


In [118]:
# hour4 df
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df.tail(5)

,time,open,high,low,close
851,2025-07-03 20:00:00,1.17524,1.17595,1.17444,1.17573
852,2025-07-04 00:00:00,1.17562,1.17715,1.17502,1.17670
853,2025-07-04 04:00:00,1.17670,1.17803,1.17560,1.17802
854,2025-07-04 08:00:00,1.17802,1.17856,1.17677,1.17746
855,2025-07-04 12:00:00,1.17746,1.17822,1.17683,1.17707


In [119]:
# daily df
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df

,time,open,high,low,close
0,2024-12-16,1.04850,1.05248,1.04746,1.05114
1,2024-12-17,1.05105,1.05343,1.04793,1.04915
2,2024-12-18,1.04907,1.05128,1.03437,1.03495
3,2024-12-19,1.03514,1.04224,1.03440,1.03629
4,2024-12-20,1.03650,1.04475,1.03431,1.04268
...,...,...,...,...,...
138,2025-06-30,1.17279,1.17880,1.17077,1.17877
139,2025-07-01,1.17868,1.18298,1.17615,1.18052
140,2025-07-02,1.18056,1.18101,1.17470,1.17982
141,2025-07-03,1.17983,1.18101,1.17172,1.17573


In [120]:
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci 
daily_df["pivot"] = ((daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3).shift(1)
daily_df["prange"] = (daily_df["high"] - daily_df["low"]).shift(1)
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df[["time","S2","R2"]].tail(10)


,time,S2,R2
133,2025-06-23,1.148329,1.155238
134,2025-06-24,1.145594,1.161786
135,2025-06-25,1.156423,1.165037
136,2025-06-26,1.159173,1.168467
137,2025-06-27,1.164189,1.175684
138,2025-06-30,1.167308,1.176392
139,2025-07-01,1.171151,1.181076
140,2025-07-02,1.175662,1.184104
141,2025-07-03,1.174610,1.182410
142,2025-07-04,1.170412,1.181895


In [121]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
#ohlc4
hourly_df['ohlc4'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4

# sma 1
hourly_df['sma_1'] = hourly_df['ohlc4'].rolling(furst_sma_period).mean()
hourly_df['prev_sma_1'] = hourly_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
hourly_df['sma_14'] = hourly_df['ohlc4'].rolling(second_sma_period).mean()
hourly_df

,time,open,high,low,close,ohlc4,sma_1,prev_sma_1,sma_14
0,2024-12-16 00:00:00,1.04850,1.05213,1.04797,1.05190,1.050125,1.050125,NaN,NaN
1,2024-12-16 04:00:00,1.05191,1.05230,1.05098,1.05196,1.051788,1.051788,1.050125,NaN
2,2024-12-16 08:00:00,1.05196,1.05248,1.05003,1.05051,1.051245,1.051245,1.051788,NaN
3,2024-12-16 12:00:00,1.05051,1.05084,1.04833,1.05078,1.050115,1.050115,1.051245,NaN
4,2024-12-16 16:00:00,1.05075,1.05207,1.04746,1.05112,1.050350,1.050350,1.050115,NaN
...,...,...,...,...,...,...,...,...,...
851,2025-07-03 20:00:00,1.17524,1.17595,1.17444,1.17573,1.175340,1.175340,1.175358,1.178291
852,2025-07-04 00:00:00,1.17562,1.17715,1.17502,1.17670,1.176122,1.176122,1.175340,1.178117
853,2025-07-04 04:00:00,1.17670,1.17803,1.17560,1.17802,1.177088,1.177088,1.176122,1.178011
854,2025-07-04 08:00:00,1.17802,1.17856,1.17677,1.17746,1.177703,1.177703,1.177088,1.177812


In [122]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
#ohlc4
daily_df['ohlc4'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
# sma 11
daily_df['sma_11'] = daily_df['ohlc4'].rolling(thurd_sma_period).mean()
daily_df['prev_sma_11'] = daily_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
daily_df['sma_114'] = daily_df['ohlc4'].rolling(fourth_sma_period).mean()
daily_df[["time","sma_11","sma_114","prev_sma_11"]]

,time,sma_11,sma_114,prev_sma_11
0,2024-12-16,1.049895,NaN,NaN
1,2024-12-17,1.050390,NaN,1.049895
2,2024-12-18,1.042417,NaN,1.050390
3,2024-12-19,1.037017,NaN,1.042417
4,2024-12-20,1.039560,NaN,1.037017
...,...,...,...,...
138,2025-06-30,1.175283,1.157283,1.171388
139,2025-07-01,1.179582,1.159724,1.175283
140,2025-07-02,1.179023,1.161468,1.179582
141,2025-07-03,1.177072,1.162981,1.179023


In [123]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('17:00:00', '%H:%M:%S').time()
end_trading_section = datetime.strptime('23:45:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
#trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]
# open one trade a day
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
# Close all open positions at 11:45 PM
open_positions = daily_df[daily_df['time'].dt.time == end_trading_section]

# function to trading
daily_df["sell_stope_loss"] = (daily_df["R2"] - daily_df["S1"]) /2 + daily_df["R2"] 
daily_df["buy_stope_loss"] = (daily_df["S2"] - ((daily_df["R1"] - daily_df["S2"]) /2))
daily_df['sell_profit'] = daily_df['low'] <= daily_df["S1"]
daily_df['buy_profit'] = daily_df["high"] >= daily_df["R1"]
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,S2,S3,ohlc4,sma_11,prev_sma_11,sma_114,sell_stope_loss,buy_stope_loss,sell_profit,buy_profit
0,2024-12-16,1.04850,1.05248,1.04746,1.05114,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.049895,1.049895,NaN,NaN,NaN,NaN,False,False
1,2024-12-17,1.05105,1.05343,1.04793,1.04915,1.050360,0.00502,1.052278,1.053462,1.055380,...,1.047258,1.045340,1.050390,1.050390,1.049895,NaN,1.055972,1.044748,True,True
2,2024-12-18,1.04907,1.05128,1.03437,1.03495,1.050170,0.00550,1.052271,1.053569,1.055670,...,1.046771,1.044670,1.042417,1.042417,1.050390,NaN,1.056319,1.044021,True,False
3,2024-12-19,1.03514,1.04224,1.03440,1.03629,1.040200,0.01691,1.046660,1.050650,1.057110,...,1.029750,1.023290,1.037017,1.037017,1.042417,NaN,1.059105,1.021295,False,False
4,2024-12-20,1.03650,1.04475,1.03431,1.04268,1.037643,0.00784,1.040638,1.042488,1.045483,...,1.032798,1.029803,1.039560,1.039560,1.037017,NaN,1.046408,1.028878,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2025-06-30,1.17279,1.17880,1.17077,1.17877,1.171850,0.00735,1.174658,1.176392,1.179200,...,1.167308,1.164500,1.175283,1.175283,1.171388,1.157283,1.180067,1.163633,False,True
139,2025-07-01,1.17868,1.18298,1.17615,1.18052,1.176113,0.00803,1.179181,1.181076,1.184143,...,1.171151,1.168083,1.179582,1.179582,1.175283,1.159724,1.185091,1.167136,False,True
140,2025-07-02,1.18056,1.18101,1.17470,1.17982,1.179883,0.00683,1.182492,1.184104,1.186713,...,1.175662,1.173053,1.179023,1.179023,1.179582,1.161468,1.187519,1.172247,True,False
141,2025-07-03,1.17983,1.18101,1.17172,1.17573,1.178510,0.00631,1.180920,1.182410,1.184820,...,1.174610,1.172200,1.177072,1.177072,1.179023,1.162981,1.185565,1.171455,True,True


In [124]:
def entring_stoploss_profit(row, ohlc_df) :
    hour = row['time'].hour
    week = row['time'].dayofweek
    if row["prev_sma_11"] > row['sma_114'] and row["low_x"] <= row["S2"]:# and (hour >= 10 and hour < 18):
        return "buy"
    elif row['prev_sma_11'] < row['sma_114'] and row["high_x"] >= row["R2"]:# and (hour >= 10 and hour < 18):
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df = pd.merge_asof(ohlc_df, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, ohlc_df = ohlc_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

,signal,time
3858,sell,2025-01-06 09:30:00
5482,sell,2025-01-14 00:50:00
5946,sell,2025-01-15 15:30:00
6541,sell,2025-01-17 17:05:00
6810,sell,2025-01-20 15:30:00
7706,buy,2025-01-23 18:10:00
8369,buy,2025-01-28 01:25:00
8786,buy,2025-01-29 12:10:00
9420,buy,2025-01-31 17:00:00
10017,sell,2025-02-04 18:45:00


In [125]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "daily_low",
    "high_y": "daily_high",
    "open_y": "daily_open",
    "close_y": "daily_close"
})


In [126]:
print(merged_df.columns.tolist())
merged_df


['time', 'open', 'high', 'low', 'close', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'pivot', 'prange', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', 'ohlc4', 'sma_11', 'prev_sma_11', 'sma_114', 'sell_stope_loss', 'buy_stope_loss', 'sell_profit', 'buy_profit', 'signal', 'is_first_signal']


,time,open,high,low,close,daily_open,daily_high,daily_low,daily_close,pivot,...,ohlc4,sma_11,prev_sma_11,sma_114,sell_stope_loss,buy_stope_loss,sell_profit,buy_profit,signal,is_first_signal
0,2024-12-16 00:00:00,1.04850,1.04863,1.04797,1.04856,1.04850,1.05248,1.04746,1.05114,NaN,...,1.049895,1.049895,NaN,NaN,NaN,NaN,False,False,None,NaN
1,2024-12-16 00:05:00,1.04856,1.04865,1.04854,1.04856,1.04850,1.05248,1.04746,1.05114,NaN,...,1.049895,1.049895,NaN,NaN,NaN,NaN,False,False,None,NaN
2,2024-12-16 00:10:00,1.04856,1.04871,1.04853,1.04862,1.04850,1.05248,1.04746,1.05114,NaN,...,1.049895,1.049895,NaN,NaN,NaN,NaN,False,False,None,NaN
3,2024-12-16 00:15:00,1.04862,1.04880,1.04857,1.04867,1.04850,1.05248,1.04746,1.05114,NaN,...,1.049895,1.049895,NaN,NaN,NaN,NaN,False,False,None,NaN
4,2024-12-16 00:20:00,1.04871,1.04876,1.04857,1.04864,1.04850,1.05248,1.04746,1.05114,NaN,...,1.049895,1.049895,NaN,NaN,NaN,NaN,False,False,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41036,2025-07-04 11:40:00,1.17735,1.17764,1.17714,1.17756,1.17562,1.17856,1.17502,1.17707,1.176153,...,1.176567,1.176567,1.177072,1.164485,1.18654,1.165767,False,False,None,NaN
41037,2025-07-04 11:45:00,1.17756,1.17760,1.17720,1.17740,1.17562,1.17856,1.17502,1.17707,1.176153,...,1.176567,1.176567,1.177072,1.164485,1.18654,1.165767,False,False,None,NaN
41038,2025-07-04 11:50:00,1.17740,1.17760,1.17726,1.17754,1.17562,1.17856,1.17502,1.17707,1.176153,...,1.176567,1.176567,1.177072,1.164485,1.18654,1.165767,False,False,None,NaN
41039,2025-07-04 11:55:00,1.17754,1.17764,1.17736,1.17746,1.17562,1.17856,1.17502,1.17707,1.176153,...,1.176567,1.176567,1.177072,1.164485,1.18654,1.165767,False,False,None,NaN


In [127]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    if 'signal' not in data:
        print("Warning: 'signal' column is missing!")
        return
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = data['S2'] - (data["S2"] - ((data["R1"] - data["S2"]) /2)) # example SL
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = ((data["R2"] - data["S1"]) /2 + data["R2"]) - data["R2"]
        volume = risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and ((data["low"] <= data["S2"] - ((data["R1"] - data["S2"]) /2))):
                                            #or (data["high"] >= data["R1"])
                                            #or (data['time'].time() == pd.to_datetime('23:00:00').time())):
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and ((data["high"] >= (data["R2"] - data["S1"]) /2 + data["R2"])):
                                            #or (data["low"] <= data["S1"])
                                            #or (data['time'].time() == pd.to_datetime('23:00:00').time())):
            orders.close_trade(trade)

In [128]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,sell,70175.438596,2025-01-06 09:30:00,1.03191,2025-01-06 11:00:00,1.03503,0,0,{},-218.95,-4.912281,-223.862281,-223.862281,9776.137719
1,closed,EURUSD,sell,55096.418733,2025-01-14 00:50:00,1.02647,2025-01-14 21:35:00,1.03025,0,0,{},-208.26,-3.856749,-212.116749,-435.97903,9564.02097
2,closed,EURUSD,sell,56737.588652,2025-01-15 15:30:00,1.03007,2025-01-20 15:30:00,1.03139,0,0,{},-74.89,-3.971631,-78.861631,-514.840661,9485.159339
3,closed,EURUSD,buy,61633.281972,2025-01-23 18:10:00,1.03964,2025-02-03 00:00:00,1.02357,0,0,{},-990.45,-4.31433,-994.76433,-1509.604991,8490.395009
4,closed,EURUSD,sell,28735.632184,2025-02-04 18:45:00,1.03823,2025-02-11 21:40:00,1.03623,0,0,{},57.47,-2.011494,55.458506,-1454.146485,8545.853515
5,closed,EURUSD,sell,44792.833147,2025-02-12 19:00:00,1.03914,2025-02-20 20:15:00,1.04949,0,0,{},-463.61,-3.135498,-466.745498,-1920.891984,8079.108016
6,closed,EURUSD,buy,74487.895717,2025-02-27 06:30:00,1.04657,2025-02-27 15:50:00,1.0446,0,0,{},-146.74,-5.214153,-151.954153,-2072.846136,7927.153864
7,closed,EURUSD,sell,66666.666667,2025-03-03 03:10:00,1.04202,2025-03-03 13:30:00,1.04504,0,0,{},-201.33,-4.666667,-205.996667,-2278.842803,7721.157197
8,closed,EURUSD,sell,34752.389227,2025-03-04 13:50:00,1.05266,2025-03-04 20:55:00,1.0587,0,0,{},-209.90,-2.432667,-212.332667,-2491.17547,7508.82453
9,closed,EURUSD,buy,73529.411765,2025-03-13 10:35:00,1.08682,2025-03-13 15:45:00,1.0839,0,0,{},-214.71,-5.147059,-219.857059,-2711.032529,7288.967471


In [129]:
mm = bt.plot_balance()
display(mm)

In [130]:
pnl_chart = bt.plot_pnl()
display(pnl_chart)

In [131]:
evaluate_backtest(bt.trades)

biggest_profit: 2276.49
daily_drawdown: -990.45
max_drawown: -2729.61
avg_win: 611.43
avg_loss: -269.71
count_profit_trades: 5
count_loss_trades: 12
Win Rate1: 29.41%
rrr: 2.27


,order_type,profit
0,buy,737.45
1,sell,-916.81


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,commission,profit_net,profit_cumulative,balance,current_max,drawdown,dayofweek,hourofday,month,year
0,closed,EURUSD,sell,70175.438596,2025-01-06 09:30:00,1.03191,2025-01-06 11:00:00,1.03503,0,0,...,-4.912281,-223.862281,-223.862281,9776.137719,-223.862281,0.0,0,9,2025-01,2025
1,closed,EURUSD,sell,55096.418733,2025-01-14 00:50:00,1.02647,2025-01-14 21:35:00,1.03025,0,0,...,-3.856749,-212.116749,-435.97903,9564.02097,-223.862281,-212.116749,1,0,2025-01,2025
2,closed,EURUSD,sell,56737.588652,2025-01-15 15:30:00,1.03007,2025-01-20 15:30:00,1.03139,0,0,...,-3.971631,-78.861631,-514.840661,9485.159339,-223.862281,-290.978381,2,15,2025-01,2025
3,closed,EURUSD,buy,61633.281972,2025-01-23 18:10:00,1.03964,2025-02-03 00:00:00,1.02357,0,0,...,-4.31433,-994.76433,-1509.604991,8490.395009,-223.862281,-1285.74271,3,18,2025-01,2025
4,closed,EURUSD,sell,28735.632184,2025-02-04 18:45:00,1.03823,2025-02-11 21:40:00,1.03623,0,0,...,-2.011494,55.458506,-1454.146485,8545.853515,-223.862281,-1230.284205,1,18,2025-02,2025
5,closed,EURUSD,sell,44792.833147,2025-02-12 19:00:00,1.03914,2025-02-20 20:15:00,1.04949,0,0,...,-3.135498,-466.745498,-1920.891984,8079.108016,-223.862281,-1697.029703,2,19,2025-02,2025
6,closed,EURUSD,buy,74487.895717,2025-02-27 06:30:00,1.04657,2025-02-27 15:50:00,1.0446,0,0,...,-5.214153,-151.954153,-2072.846136,7927.153864,-223.862281,-1848.983856,3,6,2025-02,2025
7,closed,EURUSD,sell,66666.666667,2025-03-03 03:10:00,1.04202,2025-03-03 13:30:00,1.04504,0,0,...,-4.666667,-205.996667,-2278.842803,7721.157197,-223.862281,-2054.980522,0,3,2025-03,2025
8,closed,EURUSD,sell,34752.389227,2025-03-04 13:50:00,1.05266,2025-03-04 20:55:00,1.0587,0,0,...,-2.432667,-212.332667,-2491.17547,7508.82453,-223.862281,-2267.313189,1,13,2025-03,2025
9,closed,EURUSD,buy,73529.411765,2025-03-13 10:35:00,1.08682,2025-03-13 15:45:00,1.0839,0,0,...,-5.147059,-219.857059,-2711.032529,7288.967471,-223.862281,-2487.170248,3,10,2025-03,2025


ValueError: Plotly Express cannot process wide-form data with columns of different type.

In [ ]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                        open = ohlc_df['open'],
                                        high = ohlc_df['high'],
                                        low = ohlc_df['low'],
                                        close = ohlc_df['close'])])

# fig.update_layout(height=600, xaxis_rangeslider_visible=False)
# 📌 Add Pivot Points as Horizontal Lines
pivot_levels = ["sell_stope_loss", "R2", "buy_stope_loss", "S2"]
colors = {'sell_stope_loss': 'red', 'buy_stope_loss': 'red', 'R2': 'green', 'S2': 'green'}


for _, row in daily_df.iterrows():
    for level in pivot_levels:
        fig.add_trace(go.Scatter(
            x=[row['time'], row['time'] + pd.Timedelta(days=1)],  # Extend for one day
            y=[row[level], row[level]],
            mode='lines',
            name=f"{level} {row['time']}",
            line=dict(color=colors[level], width=1.5, dash='dot')
        ))

# 📌 Customize Layout
fig.update_layout(
    title="Pivot Points on Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)
fig